## This notebook is for testing accuracy of our text model, as we may not fine-tune any model

In [ ]:
!pip install torch
!pip install transformers

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
#we get only messages_2 as we already have around 70k messages which is enough for testing our accuracy
messages_2 = pd.read_csv("cleaned_messages_2.csv")

In [ ]:
import numpy as np
#lets split into offensive vs non offensive based wheter the risk is higher or lower 5
bad = messages_2[messages_2['risk'] >= 5.0]['raw_message'].values
good = messages_2[messages_2['risk'] <= 1.0]['raw_message'].iloc[0:10967].values
data = np.concatenate((bad,good),axis=0)
labels = np.concatenate((np.full(len(bad),1), np.full(len(bad),0)), axis=0)

In [ ]:
#shuffle labels and data the same way
from sklearn.utils import shuffle
data, labels = shuffle(data, labels)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report

#Testing the accuracy

# Load the model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-offensive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set batch size and split the inputs into batches
batch_size = 32
num_batches = len(data) // batch_size + 1
batches = [data[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# Generate predictions for each batch of inputs
predicted_labels = []
for batch in batches:
    # Tokenize the batch
    tokenized_inputs = tokenizer(batch.tolist(), padding=True, truncation=True, return_tensors="pt")

    # Generate predictions for the batch
    with torch.no_grad():
        logits = model(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])[0]
        predicted_labels_batch = torch.argmax(logits, axis=1).tolist()
    
    predicted_labels += predicted_labels_batch

# Print a classification report to evaluate model performance
print(classification_report(labels, predicted_labels))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

           0       0.72      0.99      0.84     10967
           1       0.98      0.62      0.76     10967

    accuracy                           0.81     21934
   macro avg       0.85      0.81      0.80     21934
weighted avg       0.85      0.81      0.80     21934



As we can see, we have a proper 81% accuracy, which is quite a good model. Because of this, we will proceed with inference on this model (in our app in Streamlit)